In [1]:
!pip install ht CoolProp fluids

Defaulting to user installation because normal site-packages is not writeable


In [2]:
%matplotlib notebook
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import ht,fluids,os
import CoolProp as cp
from IPython.display import Latex
from matplotlib.lines import Line2D
MK = Line2D.markers
mk = [mki for mki in MK.keys()][2:]

In [33]:
T_ac_in1= 175
T_ac_in2 = 89
T_ac_out1 = T_ac_in2
T_ac_out2 = 46
T_w_in1 = 20
T_w_out2 = 80

m_ac = 6

T_wm1 = (T_w_in1+T_ac_in2)/2

Cp_w = cp.CoolProp.PropsSI('C','T',T_wm1+273,'P',101.3e3,'Water')
Pr_w1 = cp.CoolProp.PropsSI('Prandtl','T',T_wm1+273,'P',101.3e3,'Water')
mu_w1 = cp.CoolProp.PropsSI('V','T',T_wm1+273,'P',101.3e3,'Water')
rho_w1 = cp.CoolProp.PropsSI('D','T',T_wm1+273,'P',101.3e3,'Water')
k_w1 = cp.CoolProp.PropsSI('L','T',T_wm1+273,'P',101.3e3,'Water')
nu_w1 = mu_w1/rho_w1
a_w1 = nu_w1 / Pr_w1

Cp_ac = 1480
C_ac = m_ac*Cp_ac

In [13]:
U1 = 52
U2 = 38

In [40]:
Q1 = Cp_ac*m_ac* (T_ac_in1-T_ac_out1)
Q2 = Cp_ac*m_ac* (T_ac_in2-T_ac_out2)

#Cp_w*m_w * (T_w_out2-T_w_out1) = Q1
#Cp_w*m_w * (T_w_out1-T_w_in1) = Q2
#Cp_w*m_w * (T_w_out2-T_w_in1) = Q1+Q2
m_w = (Q1+Q2)/(Cp_w*(T_w_out2-T_w_in1))
T_w_out11 = Q2/(Cp_w*m_w  )+T_w_in1

In [44]:
C_ac

8880

In [43]:
print(Q1/1e3,Q2/1e3)

763.68 381.84


In [58]:
Qmax1 = (T_ac_in1-T_w_out11)*C_ac

Qmax2 = (T_ac_in2-T_w_in1)*C_ac


eps1 = Q1/Qmax1
eps2 = Q2/Qmax2
print(eps1,eps2)
Cr = C_ac / Cp_w/m_w

0.6370370370370371 0.6231884057971014


In [64]:
Nut1 = ht.NTU_from_effectiveness(eps1,Cr)
Nut2 = ht.NTU_from_effectiveness(eps2,Cr)

A1 = Nut1 *U1/ (C_ac)
A2 = Nut2 *U2/ C_ac
print(A1,A2)

0.007248132310240152 0.005070038307802891


In [65]:
Nut1

1.2377579791333182

In [232]:
hrad = ((T_c+273)+(T_a+273))* ((T_c+273)**2+(T_a+273)**2) *sigma_sb*epsilon_c
Qrad = sigma_sb*((T_c+273)**4-(T_a+273)**4)*L * 2
Qrad2 = hrad*(T_c-T_a)*2*L

htotal = (hrad**-1+hconvf**-1)**-1
display(Latex(f'$h_{{total}}={htotal:.1f}$W/m$^2$'))

<IPython.core.display.Latex object>

In [191]:
from scipy.special import erfc

In [192]:
def theta (x,a,t,hc,k):
    z = x/(4*a*t)**0.5
    z2 = hc/k*(a*t)**0.5
    arg = hc*x/k+(hc/k)**2*a*t
    return erfc(z)-np.exp(arg)*erfc(z+z2)
    